In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import lxml.html as lh

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#Get the HTML of the Wiki page, convert into a table with help of read_html (read HTML tables into a list of DataFrame objects)
wikipedia_list_codeposttal_canada = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wikipedia_list_codeposttal_canada)

toronto_codepostal_data = pd.read_html(wikipedia_page.content, header=0)[0]

toronto_data = toronto_codepostal_data[toronto_codepostal_data.Borough != 'Not assigned']

toronto_data.replace('Not assigned',toronto_data.Borough,inplace=True)

url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)

toronto_data = pd.merge(toronto_data, df_geo, on='Postal Code')
toronto_data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Make sure that you have created a Foursquare developer account and have your credentials handy

In [3]:

CLIENT_ID = 'MV5Q2OXQUNSGWGEEAHDBLKOVKPIZDX5J3BZ5DSZ1BLI5GTPI' # your Foursquare ID
CLIENT_SECRET = '0MDOUQJ1NGUY04IRB2HLMAP3C14T4OX4OGNM1AJCILYDT4IH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: MV5Q2OXQUNSGWGEEAHDBLKOVKPIZDX5J3BZ5DSZ1BLI5GTPI
CLIENT_SECRET:0MDOUQJ1NGUY04IRB2HLMAP3C14T4OX4OGNM1AJCILYDT4IH


# Get the latitude and longitude values of Toronto, Canada

In [4]:
address = 'Toronto, Canada'

# with  geopy library we get the latitude and longitude values of Toronto, Canada

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


43.6534817 -79.3839347


# Generate maps to visualize your neighborhoods and how they cluster together.

In [7]:
# create map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for latitude, longitude, Borough, Neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

